In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as mpl

In [ ]:
start = '2022-01-01'
end = '2023-12-31'
stock = 'GOOG'
data = yf.download(stock,start,end)

In [ ]:
data

In [4]:
ma_100_days = data.Close.rolling(100).mean()

In [ ]:

mpl.figure(figsize=(8,6))
mpl.plot(ma_100_days, 'r')
mpl.plot(data.Close,'g')
mpl.show()

In [6]:
ma_200_days = data.Close.rolling(200).mean()

In [ ]:
mpl.figure(figsize=(8,6))
mpl.plot(ma_100_days, 'r')
mpl.plot(ma_200_days, 'b')
mpl.plot(data.Close, 'g')
mpl.show()

In [8]:
data.dropna(inplace=True)

In [9]:
data_train = pd.DataFrame(data.Close[0:int(len(data)*0.80)])
data_test = pd.DataFrame(data.Close[int(len(data)*0.80):len(data)])

In [ ]:
data_train.shape[0]

In [ ]:
data_test.shape[0]

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

In [13]:
data_train_scale = scaler.fit_transform(data_test)

In [14]:
x = []
y = []
for i in range(100, data_train_scale.shape[0]):
    x.append(data_train_scale[i - 100:i])
    y.append(data_train_scale[i,0])

In [15]:
x, y = np.array(x), np.array(y)

In [16]:
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=True, 
               input_shape =((x.shape[1],1))))
model.add(Dropout(0.2))
model.add(LSTM(units=60,activation='relu',return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=80,activation='relu',return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1))

In [18]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x,y, epochs=50, batch_size=32, verbose=1)

In [ ]:
model.summary()

In [21]:
pas_100_days = data_train.tail(100)

In [22]:
data_test = pd.concat([pas_100_days, data_test],ignore_index = True)

In [23]:
data_test_scale = scaler.fit_transform(data_test)

In [24]:
x = []
y = []
for i in range(100, data_train_scale.shape[0]):
    x.append(data_train_scale[i - 100:i])
    y.append(data_train_scale[i,0])
x,y = np.array(x), np.array(y)

In [ ]:
y_predict = model.predict(x)

In [26]:
scale = 1/scaler.scale_

In [27]:
y_predict = y_predict*scale

In [28]:
y =  y*scale

In [ ]:
mpl.figure(figsize=(10,8))
mpl.plot(y_predict,'r', label = 'predicted price')
mpl.plot(y,'g',label = 'original price')
mpl.xlabel('time')
mpl.ylabel('price')
mpl.legend()
mpl.show()

In [30]:
model.save('Stock Predictions Model.keras')